In [5]:
#first version which afterwards was later on improved to an more efficient python script (deplete_p_loop.py (version2) --> deplete_p_loop_faster.py (version3)))
import sys
!{sys.executable} -m pip install regex

import regex
import pandas as pd
import os

You should consider upgrading via the 'C:\Users\JonathanG03\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached regex-2021.8.3-cp39-cp39-win_amd64.whl (270 kB)


In [6]:
DIR = r'D:\Dropbox\MotifAi_Exercises\Exp3_HCV_Naive+Exp7_8_HIV\reads_filtration'

In [21]:
list_sequences = []
rpm = []

with open(r"C:\Users\JonathanG03\Dropbox\MotifAi_Exercises\Exp3_HCV_Naive+Exp7_8_HIV\reads_filtration_depleted\HIV_unique_reads\EC_HIV_060_1\EC_HIV_060_1_unique_rpm.faa", 'r') as f:  

        n=0
        m=1

        for i, line in enumerate(f):
            if i == n:
                rpm.append(line)
                n+=2             
            if i == m:
                list_sequences.append(line[:-1])
                m+=2
            #if i>10002:
                #break
            
print(len(list_sequences))
print(len(rpm))

df_read = pd.DataFrame(
    {'sequence_read': list_sequences,
    'rpm': rpm
    })
df_read.head(20)

550
550


,sequence_read,rpm
0,PAHHCQQLFL,>seq_1_lib_10_len_10_counts_534051.4763800481\n
1,FPTYNFSRVR,>seq_2_lib_10_len_10_counts_379356.22591701616\n
2,QLPRFQTMVS,>seq_3_lib_10_len_10_counts_57341.45147212228\n
3,WTLTTSVQSTVA,>seq_4_lib_12_len_12_counts_1766.9725161262852\n
4,CLKVSSYGPQLC,>seq_5_lib_C10C_len_12_counts_489.6429863964405\n
5,CSEQLILFEC,>seq_6_lib_C8C_len_10_counts_468.3541609009431\n
6,PAHHCQQLFR,>seq_7_lib_10_len_10_counts_457.70974815319437\n
7,PAHNCQQLFL,>seq_8_lib_10_len_10_counts_457.70974815319437\n
8,PANHCQQLFL,>seq_9_lib_10_len_10_counts_457.70974815319437\n
9,PAHHCQQLLL,>seq_10_lib_10_len_10_counts_447.0653354054457\n


In [22]:
def find_rpm(rpm_line):
    matches = regex.search('(counts_)',rpm_line)
    index_end = matches.end()
    return index_end

df_read['rpm_end']=df_read['rpm'].apply(find_rpm)




In [23]:
df_read['rpm'] = df_read.apply(lambda x: x['rpm'][x['rpm_end']:-1], 1)
df_read


,sequence_read,rpm,rpm_end
0,PAHHCQQLFL,534051.4763800481,28
1,FPTYNFSRVR,379356.22591701616,28
2,QLPRFQTMVS,57341.45147212228,28
3,WTLTTSVQSTVA,1766.9725161262852,28
4,CLKVSSYGPQLC,489.6429863964405,30
...,...,...,...
545,YPTYNFSMVR,10.644412747748706,30
546,YPTYNFSRDR,10.644412747748706,30
547,YPTYNFTRVR,10.644412747748706,30
548,YPTYNISGVR,10.644412747748706,30


In [24]:
df_read['rpm']=df_read['rpm'].astype(str).astype(float)

In [25]:
print (df_read['rpm'].sum())
assert 999999<df_read['rpm'].sum()<1000001, "sum of RPM is not 1000000"

1000000.0


In [26]:
def find_pentameric_loop(sequence):
    matches = regex.search('(C.G[RK][LM].C)', sequence)
    if matches !=None:
        return ('pentameric loop')

df_read['pentameric_loop'] = df_read['sequence_read'].apply(find_pentameric_loop)
df_read

,sequence_read,rpm,rpm_end,pentameric_loop
0,PAHHCQQLFL,534051.476380,28,None
1,FPTYNFSRVR,379356.225917,28,None
2,QLPRFQTMVS,57341.451472,28,None
3,WTLTTSVQSTVA,1766.972516,28,None
4,CLKVSSYGPQLC,489.642986,30,None
...,...,...,...,...
545,YPTYNFSMVR,10.644413,30,None
546,YPTYNFSRDR,10.644413,30,None
547,YPTYNFTRVR,10.644413,30,None
548,YPTYNISGVR,10.644413,30,None


In [27]:
df_read['pentameric_loop'].value_counts()

pentameric loop    11
Name: pentameric_loop, dtype: int64

In [28]:
groups=df_read.groupby('pentameric_loop')
print(groups)

In [29]:
df_rpm_pentameric_loop=df_read.groupby('pentameric_loop')['rpm'].apply(sum)
df_rpm_pentameric_loop

pentameric_loop
pentameric loop    180.955017
Name: rpm, dtype: float64

In [30]:
df_rpm_pentameric_loop[0]

180.955016711728

In [31]:
for group in groups:
     for item in group:
         if type(item) !=str:
             df_new=item

df_new
list_of_sequences_for_depletion=df_new['sequence_read'].tolist()
print(list_of_sequences_for_depletion)

['CLGKLGC', 'CGGKLHC', 'CLGRMVC', 'CCVGKLFCSDSSYC', 'CEGRLRCKPHVHDC', 'CHGKLAC', 'CIGRMVC', 'CLGRLTC', 'CSGRLRC', 'CTGRLTCF', 'CVGKMLCNPS']


In [33]:
from Bio import SeqIO
fasta_sequences = SeqIO.parse(r"C:\Users\JonathanG03\Dropbox\MotifAi_Exercises\Exp3_HCV_Naive+Exp7_8_HIV\reads_filtration_depleted\HIV_unique_reads\EC_HIV_060_1\EC_HIV_060_1_unique_rpm.faa",'fasta')

In [34]:
depleted_list=[]

for fasta_entry in fasta_sequences:
    if fasta_entry.seq not in list_of_sequences_for_depletion:
        depleted_list.append(fasta_entry)


with open(r"C:\Users\JonathanG03\Dropbox\MotifAi_Exercises\Exp3_HCV_Naive+Exp7_8_HIV\reads_filtration_depleted\HIV_unique_reads\EC_HIV_060_1\EC_HIV_060_1_unique_rpm"+"_depleted.faa", "w") as output_handle:
    SeqIO.write(depleted_list, output_handle, "fasta")